In [63]:
import numpy as np
import pandas as pd
import mlflow
import dagshub

In [64]:
dagshub.init(repo_owner='akshatsharma2407', repo_name='GMC_motors', mlflow=True)

mlflow.set_tracking_uri('https://dagshub.com/akshatsharma2407/GMC_motors.mlflow')

Initialized MLflow to track repo "akshatsharma2407/GMC_motors"

Repository akshatsharma2407/GMC_motors initialized!

In [65]:
mlflow.autolog()
mlflow.set_experiment(experiment_name='GMC_exp_OE_TARGET')
mlflow.start_run()

2025/03/09 20:22:05 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/03/09 20:22:05 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2025/03/09 20:22:05 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


<ActiveRun: >

In [66]:
df = pd.read_csv('C:/Users/aksha/Downloads/CLEANED_GMC_DIESEL.csv')

In [67]:
df.drop(columns=['PRICE RANGE','MAKE ORIGIN','PARENT COMPANY','IMAGE','BRAND'],inplace=True)

In [68]:
df['AGE OF CAR'] = df['AGE OF CAR'].astype(str)
df['MODEL'] = df['MODEL'].astype(str)

In [69]:
df.dropna(inplace=True)

In [70]:
df.drop_duplicates(inplace=True)

In [71]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression

In [72]:
xtrain,xtest,ytrain,ytest = train_test_split(df.drop(columns=['PRICE($)']),df['PRICE($)'],random_state=42,test_size=0.2)

In [73]:
ct1 = ColumnTransformer(
    [
        ('RatingImputer',SimpleImputer(missing_values=-1,strategy='mean'),['RATING']),
        ('OHE',ce.TargetEncoder(verbose=1),['CAR NAME','MODEL/CLASS','DEALER NAME','DEALER LOCATION (CITY)','DEALER LOCATION (STATE)']),
        ('OE',OrdinalEncoder(categories=
                             [
                                 ["1937", "1951", "1952", "1966", "1968", "1977", "1979", "1984", "1986", "1987", 
        "1988", "1989", "1996", "1998", "1999", "2000", "2001", "2002", "2003", "2004", 
        "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", 
        "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023", "2024"],
       ['Used','GMC Certified','New'],
       ["87", "73", "72", "58", "56", "47", "45", "40", "38", "37", "36", "35", "28", 
        "26", "25", "24", "23", "22", "21", "20", "19", "18", "17", "16", "15", "14", 
        "13", "12", "11", "10", "9", "8", "7", "6", "5", "4", "3", "2", "1", "0"]
       ]
       ),['MODEL','STOCK TYPE','AGE OF CAR'])
    ],
    remainder='passthrough'
)

In [74]:
ct2 = ColumnTransformer(
    [
        ('stdscaler',StandardScaler(),slice(0,820))
    ]
)

In [75]:
pipe = Pipeline([
    ('ct1',ct1),
    ('ct2',ct2)
])

In [76]:
pipe.set_output(transform='pandas')
xtrain_trans = pipe.fit_transform(xtrain,ytrain)

2025/03/09 20:22:07 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\aksha\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/03/09 20:22:07 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\aksha\AppData\Lo

🏃 View run handsome-calf-848 at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/3/runs/7cdeaf39891b4ce0b69ea529a82814fa
🧪 View experiment at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/3
🏃 View run redolent-ant-930 at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/3/runs/591123d049a14b688bf2ac87b50ddc33
🧪 View experiment at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/3
🏃 View run efficient-smelt-704 at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/3/runs/622750b876ec4bdda183616e1f6ea676
🧪 View experiment at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/3


In [ ]:
xtest_trans = pipe.transform(xtest)

2025/03/09 20:22:22 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\aksha\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [79]:
param_grids = {
    'LinearRegression': {},
    'DecisionTreeRegressor': {
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10]
    },
    'SVR': {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf']
    },
    'RandomForestRegressor': {
        'n_estimators': [50, 100, 200],
        'max_features': ['auto', 'sqrt']
    }
}

In [80]:
#baseline models
models = {
    'LinearRegression' : LinearRegression(),
    'DecisionTreeRegressor' : DecisionTreeRegressor(),
    'SVR' : SVR(),
    'RandomForestRegressor' : RandomForestRegressor()
}

In [81]:
for i,j in models.items():
    print('training ',i,'....')
    j.fit(xtrain_trans,ytrain)
    ypred = j.predict(xtest_trans)
    print('mean squared error',mean_squared_error(ytest,ypred))
    print('mean_absolute_error',mean_absolute_error(ytest,ypred))
    print('r2_score',r2_score(ytest,ypred))
    print('='*20)

    print('\n\n\n')

training  LinearRegression ....
mean squared error 74003180.71016759
mean_absolute_error 6648.502032259483
r2_score 0.8087998488130238




training  DecisionTreeRegressor ....
mean squared error 63384733.60510302
mean_absolute_error 5379.172126098942
r2_score 0.8362344627360482




training  SVR ....
mean squared error 375780147.1355449
mean_absolute_error 15916.254986756925
r2_score 0.02910631332487179




training  RandomForestRegressor ....


2025/03/09 20:23:45 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 64f43667ca354f81bb0c506f74a90828. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]


mean squared error 47797330.33456283
mean_absolute_error 4634.859663829156
r2_score 0.876507243356276






In [82]:
# hyperparameter tuning

for i,j in models.items():
    print('training ',i,'....')
    grid_search = GridSearchCV(j,param_grid=param_grids[i],cv=5,verbose=1,n_jobs=-1,scoring='r2')

    grid_search.fit(xtrain_trans,ytrain)

    print(f'best params for {i} is : ',grid_search.best_params_)
    print(f'best params for {j} is : ',grid_search.best_score_)

    print('='*20,'\n\n')

training  LinearRegression ....
Fitting 5 folds for each of 1 candidates, totalling 5 fits


2025/03/09 20:24:04 INFO mlflow.sklearn.utils: Logging the 5 best runs, no runs will be omitted.


🏃 View run aged-dog-241 at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/3/runs/7ac7334537854238b869887c2213d80c
🧪 View experiment at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/3


2025/03/09 20:24:06 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 64f43667ca354f81bb0c506f74a90828. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]


best params for LinearRegression is :  {}
best params for LinearRegression() is :  0.8225787709335352


training  DecisionTreeRegressor ....
Fitting 5 folds for each of 9 candidates, totalling 45 fits


...

best params for SVR is :  {'C': 10, 'kernel': 'linear'}
best params for SVR() is :  0.80440307424129


training  RandomForestRegressor ....
Fitting 5 folds for each of 6 candidates, totalling 30 fits


c:\Users\aksha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\aksha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\aksha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
  File "c:\Users\aksha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py"

🏃 View run caring-calf-244 at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/3/runs/29b42aa791344d53a0c8eef2fa79d801
🧪 View experiment at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/3
🏃 View run adorable-mare-853 at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/3/runs/0522da201b3041d8823868f6acaabf71
🧪 View experiment at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/3
🏃 View run wistful-vole-7 at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/3/runs/ef84cdbec7ee43ed965c7f61ebe407d2
🧪 View experiment at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/3
🏃 View run big-goat-122 at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/3/runs/083ee707d3a24d29ab6e74f452922861
🧪 View experiment at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/3


2025/03/09 20:31:52 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 64f43667ca354f81bb0c506f74a90828. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]


best params for RandomForestRegressor is :  {'max_features': 'sqrt', 'n_estimators': 200}
best params for RandomForestRegressor() is :  0.8827113448038484




In [83]:
best_estimator = grid_search.best_estimator_

In [84]:
ypred = best_estimator.predict(xtest_trans)

In [85]:
print('mean squared error',mean_squared_error(ytest,ypred))
print('mean_absolute_error',mean_absolute_error(ytest,ypred))
print('r2_score',r2_score(ytest,ypred))

mean squared error 47005216.69515579
mean_absolute_error 4624.5725490164195
r2_score 0.8785538073844501


In [86]:
mlflow.end_run()

🏃 View run clumsy-mare-257 at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/3/runs/64f43667ca354f81bb0c506f74a90828
🧪 View experiment at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/3
